 # Exercise Sheets BCI

 ## Good Afternoon all together.
 Todays obejctive is, to write a random forrest classifier. Just that the classifier is already written.
 Hence you objectives are:
 - In this code some improvements can be made in respect to programming paradigms.
 - Also please explain for every code snippet whats happening.
 - Then visualise one EEG chanel with matplotlib
 - Last find the missing part in the code and correct it

 Good Luck and much fun!

In [ ]:
import pandas as pd
import numpy as np
from scipy import signal
import pywt
from scipy.stats import entropy
import plotly.express as px
import math
import neurokit2 as nk

df = pd.read_csv('20200605_2030.csv')
df

In [ ]:
df["Event Id"].dropna() # Well. darn it no other labels to use / Events

 This creates a 2d list of the dataframe, prints the length of the first dimension and then prints the shape.

In [ ]:
df.shape

 Are these "labels" ? Why are they using Channel 4?

In [ ]:
awakedf = df[:37502]
tireddf = df[308386:345889]



In [ ]:
awake = awakedf["Channel 4"] # Only use Channel 4?
tired = tireddf["Channel 4"] # Okay
entropy_awake = nk.entropy_sample(awake) # Check the "variablity" of the series
print(entropy_awake)
entropy_tired = nk.entropy_sample(tired) # Check the "variablity" of the series or the "sample entropy"
print(entropy_tired)
print(len(awake))
entropy_awake = nk.entropy_approximate(awake) # same as above but faster.
print(entropy_awake)
entropy_tired = nk.entropy_approximate(tired)
print(entropy_tired)


 Of interrest is that the entropy is smaller on tired probants

In [ ]:
# Generator for chunks
def datasetChunkGenerator(df, start, chunksize):
    index = 0
    while index + chunksize < len(df):
        yield df[index:index+chunksize]
        index += chunksize



In [ ]:
entropy_sample=[]
entropy_approximate=[]

for chunk in datasetChunkGenerator(df["Channel 4"], 0, 10000):
    entropy_sample.append(nk.entropy_sample(chunk))
    entropy_approximate.append(nk.entropy_approximate(chunk))



In [ ]:
fig = px.histogram(df, y="Channel 4", hover_data=df.columns)
fig.show()

In [ ]:
fig = px.line(df, y="Channel 4")
fig.show()

In [ ]:
labels = [1,1,1,1,1,0,1,1,1,1,1,1,1,1,0,0,0,0,1,0]
print(len(labels)) # labels.. not sure if they fit for the actual entropy stuff or the overwritten stuff
# A big problem I see here is that the Data is heavily biased. As a Classifier I'd just always say 1.


In [ ]:
data = pd.DataFrame({
    'sample':entropy_sample,
    'approximate':entropy_approximate
}) # why use both? They represent a similar thing
fig = px.line(data)
fig.show()



In [ ]:
# Import train_test_split function
from sklearn.model_selection import train_test_split

# Needed to be adapted to use the labels.. but who knows what those labels correspond too
X = data[['sample','approximate']][:20]  # Features ... not sure why we used sample and the approximate
y = labels  # Labels

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test



In [ ]:
for x,y,z in zip(X_train["sample"],X_train["approximate"],y_train):
    print(x,y,z)



In [ ]:
for x,y,z in zip(X_test["sample"],X_test["approximate"],y_test):
    print(x,y, z)

# Here we can see that we lost the entropy -> tiredness corralation ... probably a problem with the labels



In [ ]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)



In [ ]:

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)



In [ ]:
y_pred=clf.predict(X_test)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))



In [ ]:
# Dummydata to check the predictor
data2 = pd.DataFrame({
    'sample':np.arange(0,1,0.01),
    'approximate':np.arange(0,1,0.01),
})
display(data2.head())



In [ ]:
y = clf.predict(data2)

fig = px.line(y)
fig.show()

# not sure what this tells me....but to much or to little entropy equals tired? 

# This changes drastically depending upon the train/test shuffle 

# If id had to guess id say that my labels are not accurate and depending on which get trained the model is drastically different